In [1]:
import pandas as pd
import pyarrow.parquet as pq 
train = pq.read_table(source='C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/train').to_pandas()
test = pq.read_table(source='C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/test').to_pandas()

In [2]:
for column in train:
    if(column == "content"):
        train[column].fillna(value="",inplace=True)
    else:
        if(train[column].isnull().sum() != 0):
            media = train[column].mean()
            train[column].fillna(value=media, inplace=True)

for column in test:
    if(column == "content"):
        test[column].fillna(value="",inplace=True)
    else:
        if(test[column].isnull().sum() != 0):
            media = test[column].mean()
            test[column].fillna(value=media, inplace=True)

In [3]:
train = train.iloc[:(8*len(train)//10)]
valid = train.iloc[(8*len(train)//10):]

X_train = train.drop(['url','timedelta','popular','shares'],axis = 1)
y_train = train["popular"]*1

X_valid = valid.drop(['url','timedelta','popular','shares'],axis = 1)
y_valid = valid["popular"]*1

X_test = test.drop(['url','timedelta','popular','shares'],axis = 1)
y_test = test["popular"]*1

In [4]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from category_encoders import TargetEncoder

In [5]:
ohe_surprise1 = OneHotEncoder()
encoded_surp = ohe_surprise1.fit_transform(X_train[["surprise1"]]).toarray()
encoded_surp_test = ohe_surprise1.transform(X_test[["surprise1"]]).toarray()
encoded_surp_valid = ohe_surprise1.transform(X_valid[["surprise1"]]).toarray()

feature_labels = np.array(ohe_surprise1.categories_).ravel()
X_train = X_train.reset_index().drop(['index','surprise1'],axis = 1).join(pd.DataFrame(encoded_surp, columns = feature_labels))
X_test = X_test.reset_index().drop(['index','surprise1'],axis = 1).join(pd.DataFrame(encoded_surp_test, columns = feature_labels))
X_valid = X_valid.reset_index().drop(['index','surprise1'],axis = 1).join(pd.DataFrame(encoded_surp_valid, columns = feature_labels))

In [6]:
vect_content = CountVectorizer(analyzer = "word", stop_words = "english", max_features = 15)
encoded_content = vect_content.fit_transform(X_train["content"].tolist()).toarray()
encoded_content_test = vect_content.transform(X_test["content"].tolist()).toarray()
encoded_content_valid = vect_content.transform(X_valid["content"].tolist()).toarray()

X_train = X_train.reset_index().drop(['index','content'],axis = 1).join(pd.DataFrame(encoded_content, columns = vect_content.get_feature_names_out()))
X_test = X_test.reset_index().drop(['index','content'],axis = 1).join(pd.DataFrame(encoded_content_test, columns = vect_content.get_feature_names_out()))
X_valid = X_valid.reset_index().drop(['index','content'],axis = 1).join(pd.DataFrame(encoded_content_valid, columns = vect_content.get_feature_names_out()))

In [7]:
te = TargetEncoder()
encoded_surp2 = te.fit_transform(X_train["surprise2"],y_train)
encoded_surp2_test = te.transform(X_test["surprise2"],y_test)
encoded_surp2_valid = te.transform(X_valid["surprise2"],y_valid.reset_index().drop(["index"],axis = 1))

X_train = X_train.reset_index().drop(['index','surprise2'],axis = 1).join(pd.DataFrame(encoded_surp2, columns = te.get_feature_names()))
X_test = X_test.reset_index().drop(['index','surprise2'],axis = 1).join(pd.DataFrame(encoded_surp2_test, columns = te.get_feature_names()))
X_valid = X_valid.reset_index().drop(['index','surprise2'],axis = 1).join(pd.DataFrame(encoded_surp2_valid, columns = te.get_feature_names()))

C:\Users\tomas\AppData\Local\Programs\Python\Python37\lib\site-packages\category_encoders\target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
C:\Users\tomas\AppData\Local\Programs\Python\Python37\lib\site-packages\category_encoders\target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


In [8]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
model = XGBClassifier()
param_grid={"learning_rate": (0.05, 0.10, 0.15),
            "max_depth": [1,2,3,4,5,6,8,9,10,11],
            "min_child_weight": [ 1, 3, 5, 7],
            "gamma":[ 0.0, 0.1, 0.2],
            'n_estimators': [200, 400, 600, 800, 1000],
            "colsample_bytree":[ 0.3, 0.4]}

In [9]:
rs_xgb = RandomizedSearchCV(estimator = model, param_distributions = param_grid, cv = 3, verbose = 2, n_jobs = -1, random_state = 42)
rs_xgb.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=Non

In [10]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
param_grid = {'bootstrap': [True, False],
              'max_depth': [1,2,3,4,5,6,8,9,10,11],
              'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'n_estimators': [200, 400, 600, 800, 1000]}

In [11]:
rs_rfc = RandomizedSearchCV(estimator = model, param_distributions = param_grid, cv = 3, verbose = True, n_jobs = -1, random_state = 42)
rs_rfc.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 8, 9,
                                                      10, 11],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000]},
                   random_state=42, verbose=True)

In [12]:
from sklearn.ensemble import VotingClassifier
rfc = rs_rfc.best_estimator_
xgbc = rs_xgb.best_estimator_

In [13]:
evc = VotingClassifier(estimators = [("rfc",rfc),("xgbc",xgbc)], voting = "soft")

In [14]:
evc.fit(X_train,y_train)

VotingClassifier(estimators=[('rfc',
                              RandomForestClassifier(max_depth=11,
                                                     min_samples_leaf=4,
                                                     n_estimators=1000,
                                                     random_state=42)),
                             ('xgbc',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.3,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=0.1,
                                            gpu_id=-1, grow_policy='depthwise',
                                            importance_ty

In [16]:
print(f'Accuracy - : {evc.score(X_train,y_train):.3f}')

Accuracy - : 0.815


In [17]:
prediccion_binaria = evc.predict(X_valid)
prediccion_binaria


array([0, 0, 0, ..., 0, 0, 0])

In [18]:
proba_preds = evc.predict_proba(X_valid)[:,1]
proba_preds

array([0.11093058, 0.1056036 , 0.05318077, ..., 0.15350488, 0.07873503,
       0.2582593 ])

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_valid, prediccion_binaria))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4586
           1       0.98      0.15      0.25      1120

    accuracy                           0.83      5706
   macro avg       0.90      0.57      0.58      5706
weighted avg       0.86      0.83      0.78      5706



In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid,proba_preds)

0.8652743598529687

In [21]:
roc_auc_score(y_test,evc.predict_proba(X_test)[:,1])

0.7292363140775816